<a href="https://colab.research.google.com/github/1hamzaiqbal/MFCLIP_acv/blob/aidan%2Fgaussianlowpass/mf_clip_initial_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 0) GPU + repo
!nvidia-smi
%cd /content
!git clone https://github.com/1hamzaiqbal/MFCLIP_acv.git
%cd MFCLIP_acv/
!git checkout aidan/gaussianlowpass


Wed Nov  5 03:31:37 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-80GB          Off |   00000000:00:05.0 Off |                    0 |
| N/A   32C    P0             52W /  400W |       0MiB /  81920MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [2]:
# 1) Deps
!pip install torch torchvision timm einops yacs tqdm opencv-python scikit-learn scipy pyyaml ruamel.yaml ignite pytorch-ignite foolbox pandas matplotlib seaborn wilds ftfy



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.9/119.9 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 343.8/343.8 kB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 68.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.2/126.2 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 753.1/753.1 kB 52.8 MB/s eta 0:00:00


These next two codeblocks are only necessary if the data is not already downloaded, @Jun feel free to ignore, just ensure that the commands for training are directed to your datafiles

In [5]:
# 2) Data (example: Oxford Pets into /content/data/oxford_pets)
from torchvision.datasets import OxfordIIITPet
from torchvision import transforms
from pathlib import Path
root = Path("/content/data/")
root.mkdir(parents=True, exist_ok=True)
_ = OxfordIIITPet(root=str(root), download=True, transform=transforms.ToTensor())
print("Oxford Pets downloaded to", root)



100%|██████████| 792M/792M [00:44<00:00, 17.7MB/s]
100%|██████████| 19.2M/19.2M [00:02<00:00, 9.21MB/s]


Oxford Pets downloaded to /content/data


In [15]:
from torchvision.datasets import Flowers102
from torchvision import transforms
from pathlib import Path

# Set up data root directory
root = Path("/content/data/")
root.mkdir(parents=True, exist_ok=True)

# Download and transform the dataset
_ = Flowers102(
    root=str(root),
    download=True,
    transform=transforms.ToTensor()
)

print("Oxford Flowers 102 downloaded to", root)

100%|██████████| 345M/345M [00:16<00:00, 20.9MB/s]
100%|██████████| 502/502 [00:00<00:00, 1.32MB/s]
100%|██████████| 15.0k/15.0k [00:00<00:00, 30.3MB/s]

Oxford Flowers 102 downloaded to /content/data


In [24]:
"""
Train the target models, only necessary if they are not already trained. Replace rn18 with whatever model you want to train.
"""
!python main.py --dataset oxford_flowers --root /content/data --dataset-config-file configs/datasets/oxford_flowers.yaml --config-file configs/trainers/CoOp/rn50.yaml --flag train_scratch \
        --num_epoch 300 \
        --target "rn18" \
        --lr 0.1 \
        --bs 128

2025-11-05 05:21:59.583802: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-05 05:21:59.601413: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762320119.622699   37474 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762320119.629136   37474 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1762320119.645869   37474 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [9]:
"""
command to train unet
"""
!python main.py \
--dataset oxford_pets \
  --root /content/data \
  --config-file configs/trainers/CoOp/rn50.yaml \
  --dataset-config-file configs/datasets/oxford_pets.yaml \
  --trainer ZeroshotCLIP \
    --flag train_unet \
    --num_epoch 90 \
    --attack unet \
    --seed 1\
    --bs 64

2025-11-05 04:04:06.097579: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-05 04:04:06.115881: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762315446.137442   10292 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762315446.144006   10292 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1762315446.160882   10292 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [17]:
"""
command to train unet on different dataset
"""
!python main.py \
--dataset oxford_flowers \
  --root /content/data \
  --config-file configs/trainers/CoOp/rn50.yaml \
  --dataset-config-file configs/datasets/oxford_flowers.yaml \
  --trainer ZeroshotCLIP \
    --flag train_unet \
    --num_epoch 90 \
    --attack unet \
    --seed 1\
    --bs 64

2025-11-05 05:05:15.325959: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-05 05:05:15.344157: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762319115.366192   32423 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762319115.372875   32423 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1762319115.390163   32423 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [23]:
"""
run eval against target on oxford_pets
"""

#Note: I don't think that target flag really works, I think eval_adv tries to evaluate against all the targets listed on line 219 in main.py by default, so control it there
!python main.py \
  --root /content/data \
  --config-file configs/trainers/CoOp/rn50.yaml \
  --dataset-config-file configs/datasets/oxford_pets.yaml \
        --flag eval_adv \
      --dataset oxford_pets \
       --target "rn18" \
      --seed 7 \
        --attack "ours"

2025-11-05 05:17:45.621021: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-05 05:17:45.639476: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762319865.661851   36208 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762319865.668380   36208 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1762319865.685612   36208 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [27]:
"""
run eval against target on oxford flowers
"""
!python main.py \
  --root /content/data \
  --config-file configs/trainers/CoOp/rn50.yaml \
  --dataset-config-file configs/datasets/oxford_flowers.yaml \
        --flag eval_adv \
      --dataset oxford_flowers \
       --target "rn18" \
      --seed 7 \
        --attack "ours"

2025-11-05 05:50:42.492207: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-05 05:50:42.510739: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762321842.532249   64706 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762321842.538836   64706 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1762321842.556014   64706 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
